# Machine Learning

##  Développement du modèle

#### Import des librairies

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn import preprocessing

#### Import du fichier de données

In [79]:
df = pd.read_csv("../Data/final_dataset.csv")

In [80]:
pd.set_option('display.max_columns', None)
df.sample(3)

,Identifiant,Titre,PrixNuitee,Latitude,Longitude,Resume,Capacite_accueil,NombreSdB,NbChambres,NbLits,Type_logement,type_propriete,Cuisine,Internet,television,produits_base,Chauffage,Climatisation,machine_laver,seche_linge,parking_sur-place,wifi,television_cable,petit_dejeuner,animaux_acceptes,pourEnfants_famille,adapte_evenements,logement_fumeur,accessibilite,Ascenseur,cheminee_interieur,Interphone,Portier,Piscine,Jacuzzi,salle_sport,Entree_24-24,Cintres,fer_repasser,seche_cheveux,espace_travail_ordi,detecteur_fumee,monoxyde_carbone_detect,kit_secours,fiche_securite,extincteur,porte_chambre_verrou,reduction_semaine,reduction_mois,surcout_voyageur_supp,frais_menage,Caution,conditions_annulation,Description,reglement_interieur,duree_minimale_sejour,Quartier,index_col,tokenized_Titre,tokenized_Resume,tokenized_Type_logement,tokenized_type_propriete,tokenized_conditions_annulation,tokenized_Description,tokenized_reglement_interieur
2324,9674884,vue sur toits,29,44.822128,-0.582153,chambre avec salle eau attenante dans avec b...,2,1.0,1,0,chambre privée,appartement,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,flexibles,chambre avec salle eau attenante dans avec b...,NaN,1,nansouty saint genès,2324,vue toits,chambre salle eau attenante balcon calme vue d...,chambre privée,appartement,flexibles,chambre salle eau attenante balcon calme vue d...,NaN
4470,13658467,studio cosy bordeaux,25,44.837035,-0.578441,mon logement est situé dans cœur historique ...,2,1.0,0,0,logement entier,appartement,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,20,0,strictes,autres remarques pour les arrivées entre pa...,non fumeur convient pas aux animaux pas fête...,3,centre ville,4472,studio cosy bordeaux,logement situé cœur historique plein centre bo...,logement entier,appartement,strictes,autres remarques arrivées entre frais arrivées...,non fumeur convient animaux fête soirée convie...
4167,13407735,centre ville logement cosy parfait état wifi,52,44.833107,-0.583101,dans centre bordeaux dans quartier tribuna...,2,1.0,0,0,logement entier,appartement,1,0,1,1,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,30,500,flexibles,logement placé sur une petite place qui donne ...,NaN,2,saint augustin tauzin alphonse dupeux,4167,centre ville logement cosy parfait état wifi,centre bordeaux quartier tribunal studio entiè...,logement entier,appartement,flexibles,logement placé petite place donne charme calme...,NaN


#### Suppression de colonnes inutiles

In [81]:
del df['Identifiant']
del df['Latitude']
del df['Longitude']

#### Suppression des colonnes Titre / Resume / Type_logement / type_propriete / conditions_annulation / Description / reglement_interieur pour garder les colonnnes tokenisées

In [82]:
del df['Titre']
del df['Resume']
del df['Type_logement']
del df['type_propriete']
del df['conditions_annulation']
del df['Description']
del df['reglement_interieur'] 

In [83]:
df.fillna('', inplace=True)

In [84]:
pd.set_option('display.max_columns', None)
df.sample(3)

,PrixNuitee,Capacite_accueil,NombreSdB,NbChambres,NbLits,Cuisine,Internet,television,produits_base,Chauffage,Climatisation,machine_laver,seche_linge,parking_sur-place,wifi,television_cable,petit_dejeuner,animaux_acceptes,pourEnfants_famille,adapte_evenements,logement_fumeur,accessibilite,Ascenseur,cheminee_interieur,Interphone,Portier,Piscine,Jacuzzi,salle_sport,Entree_24-24,Cintres,fer_repasser,seche_cheveux,espace_travail_ordi,detecteur_fumee,monoxyde_carbone_detect,kit_secours,fiche_securite,extincteur,porte_chambre_verrou,reduction_semaine,reduction_mois,surcout_voyageur_supp,frais_menage,Caution,duree_minimale_sejour,Quartier,index_col,tokenized_Titre,tokenized_Resume,tokenized_Type_logement,tokenized_type_propriete,tokenized_conditions_annulation,tokenized_Description,tokenized_reglement_interieur
1532,50,4,1.0,1,2,1,1,1,1,1,0,1,0,1,1,1,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,15,30,0,0,450,1,chartrons grand parc jardin public,1532,cœur chartrons parking,beau petit studio cœur chartrons place parking...,logement entier,appartement,strictes,logement appartement très belle immeuble pierr...,heure entrée lieux appartement tout jours trou...
1642,36,2,1.0,1,1,1,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,5,100,4,nansouty saint genès,1642,logement chaleureux jardin,appartement rez chaussez petit immeuble jardin...,logement entier,appartement,modérées,quartier commerces proximité situé environ mar...,entrée lieux fait après logement agréable chal...
2666,18,1,1.0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,15,25,0,0,0,6,nansouty saint genès,2666,studio meublé bordeaux centre,studio meublé rez chaussée résidence sécurisée...,logement entier,appartement,flexibles,logement studio meublé rez chaussée résidence ...,non fumeur convient animaux fête soirée


#### Nuages de mots

In [85]:
#for column in df:
    #if df[column].dtypes == 'object':
        #print(column,  ' : \n')
        
        #text = " ".join(review for review in df[column].astype(str))
        
        #wordcloud = WordCloud(collocations=False, width = 800, height = 800, 
            #background_color ='white', 
            #min_font_size = 10).generate(text)

        # plot the WordCloud image                        
        #plt.figure(figsize = (8, 8), facecolor = None) 
        #plt.imshow(wordcloud) 
        #plt.axis("off") 
        #plt.tight_layout(pad = 0) 

        #url_image_mot = "../images/wordcloud/" + column + ".jpg"
        #plt.savefig(url_image_mot)
        #plt.show() 

#### Modèle d'apprentissage

In [86]:
X = df[['Quartier','NombreSdB', 'NbChambres', 'NbLits','tokenized_Type_logement','tokenized_type_propriete', 'Capacite_accueil', 'parking_sur-place', 'Caution']]
y = df['PrixNuitee']

In [87]:
lbl = preprocessing.LabelEncoder()
X.loc[:,'tokenized_Type_logement'] = lbl.fit_transform(X.loc[:,'tokenized_Type_logement'].astype(str))
X.loc[:,'tokenized_type_propriete'] = lbl.fit_transform(X.loc[:,'tokenized_type_propriete'].astype(str))
X.loc[:,'Quartier'] = lbl.fit_transform(X.loc[:,'Quartier'].astype(str))

/Users/thomasjaulgey/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [43]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=105)

In [45]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.33, learning_rate = 0.3, max_depth = 7, alpha = 5, n_estimators = 10)

In [46]:
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
print(preds)

[ 55.93413   55.584423  54.7823   ...  81.78904  119.29719  122.74173 ]


In [47]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE:", round(rmse, 2))
print(round(rmse, 2)/(df['PrixNuitee'].max() - df['PrixNuitee'].min()))

RMSE: 38.38
0.04322072072072072


In [48]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(xg_reg, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = abs(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 24.126 (1.303)


In [49]:
#############
#############
#############
#############

Prédiction

In [102]:
df_predition = df[['Quartier','NombreSdB', 'NbChambres', 'NbLits','tokenized_Type_logement','tokenized_type_propriete', 'Capacite_accueil', 'parking_sur-place', 'Caution']][0:0]

In [104]:
df2 = { 'Quartier'                  : 'Centre ville',
        'NombreSdB'                 : 1, 
        'NbChambres'                : 2, 
        'NbLits'                    : 2,
        'tokenized_Type_logement'   : 'logement entier',
        'tokenized_type_propriete'  : 'appartement', 
        'Capacite_accueil'          : 4, 
        'parking_sur-place'         : 1, 
        'Caution'                   : 140
        }


df_predition = df_predition.append(df2, ignore_index = True)

In [105]:
df_predition.head()

,Quartier,NombreSdB,NbChambres,NbLits,tokenized_Type_logement,tokenized_type_propriete,Capacite_accueil,parking_sur-place,Caution
0,Centre ville,1.0,2,2,logement entier,appartement,4,1,140


In [107]:
lbl = preprocessing.LabelEncoder()
df_predition.loc[:,'tokenized_Type_logement'] = lbl.fit_transform(df_predition.loc[:,'tokenized_Type_logement'].astype(str))
df_predition.loc[:,'tokenized_type_propriete'] = lbl.fit_transform(df_predition.loc[:,'tokenized_type_propriete'].astype(str))
df_predition.loc[:,'Quartier'] = lbl.fit_transform(df_predition.loc[:,'Quartier'].astype(str))

In [108]:
prediction = xg_reg.predict(df_predition)

In [127]:
print('prédiction de', prediction[0], '€')

prédiction de 64.40409 €
